<a href="https://colab.research.google.com/github/Shahidulla-Vantela/Table_extractor/blob/main/Table_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fitz
!pip install tabula-py
!pip install pymupdf

  Using cached tabula_py-2.10.0-py3-none-any.whl.metadata (7.6 kB)
Using cached tabula_py-2.10.0-py3-none-any.whl (12.0 MB)


In [2]:
import pymupdf
import fitz
import tabula
import pandas as pd
import re


def check_pages_for_tables(pdf_path):
    # Open the PDF file
    num_pages = 2
    pages_with_tables = []
    pages_without_tables = []
    for page in range(1, num_pages + 1):
        with fitz.open(pdf_path) as pdf:
            tables = tabula.read_pdf(pdf_path, pages=page, multiple_tables=True)
            if tables:
                pages_with_tables.append(page)
            else:
                pages_without_tables.append(page)
    return pages_with_tables, pages_without_tables


def extract_tables_from_pdf(pdf_path,pages):
    # Extract tables into a list of DataFrames and their corresponding areas
    tables = tabula.read_pdf(pdf_path, pages=pages, multiple_tables=False, output_format='json')
    return tables

def exclude_table_text_from_pdf(pdf_path, tables):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    non_table_text = ""
    #print(pdf_document)

    page = pdf_document[0]
    #print(page)

    # Create a mask to cover table areas
    #print(tables)
    for table in tables:
        x1 = table['left']
        y1 = table['top']
        x2 = table['left']+table['width']
        y2 = table['top']+table['height']

        # Create a rectangle based on the bounding box
        rect = fitz.Rect(x1, y1, x2, y2)
        page.add_redact_annot(rect, fill=(0, 0, 0))

    # Apply redactions to the page
    page.apply_redactions()

    # Extract the text excluding redacted areas
    non_table_text += page.get_text() + "\n"

    return non_table_text


def extract_text_from_pdf(pdf_path,pages_without_tables):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in pages_without_tables:
        page = doc[page_num-1]
        text += page.get_text("text")
    return text


def extract_table_details_gst(pdf_path,pages_with_tables):
    doc = pymupdf.Document(pdf_path)
    for page in pages_with_tables:
        page_number =doc[page-1]
        tabs = page_number.find_tables()
        tab = tabs[0]
        df = tab.to_pandas()
        df.to_csv(f"table_{page}.csv",index=False)
    return df

# Define the PDF path
pdf_path = "/content/1717391099642POCb4pwoeaW29Y96.pdf"

pages_with_tables, pages_without_tables = check_pages_for_tables(pdf_path)

df = extract_table_details_gst(pdf_path,pages_with_tables)

# Extract tables from the PDF
tables = extract_tables_from_pdf(pdf_path,pages_with_tables)
#print(tables)

# Extract non-table text from the PDF
non_table_text = exclude_table_text_from_pdf(pdf_path, tables)
text = extract_text_from_pdf(pdf_path,pages_without_tables)

entire_string = non_table_text + text

Dec 10, 2024 6:51:58 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Dec 10, 2024 6:51:58 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Dec 10, 2024 6:51:58 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 10, 2024 6:51:58 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 10, 2024 6:51:58 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Dec 10, 2024 6:52:00 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 10, 2024 6:52:00 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Dec 10, 2024 6:52:06 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 10, 2024 6:52:06 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

